# CUDA and [RAPIDS](https://developer.nvidia.com/rapidshttps://developer.nvidia.com/rapids) Tests

## Iris Database

## Test Performance

### Using GPU only

In [13]:
import time
from sklearn.datasets import make_blobs
from sklearn.manifold import TSNE
from cuml.manifold import UMAP

# Create a large dataset
n_samples = 1_000_000
n_features = 20
X, _ = make_blobs(n_samples=n_samples, n_features=n_features)

print(f'GPU: Testing with [{n_samples:,.0f}] samples and [{n_features:,.0f}] features !')

# Run UMAP with cuML (GPU)
print('BEGIN [UMAP (cuML, GPU)] ...')
start = time.time()
umap = UMAP(n_components=2)
X_transformed_cuml = umap.fit_transform(X)
end = time.time()
print(f"END [ UMAP (cuML, GPU)]. Execution time: {(end - start):.3f} seconds")

ImportError: Numba needs NumPy 1.24 or less

### Using CPU/GPU

In [10]:
import time
from sklearn.datasets import make_blobs
from sklearn.manifold import TSNE
from cuml.manifold import UMAP

# Create a large dataset
n_samples = 10_000
n_features = 20
X, _ = make_blobs(n_samples=n_samples, n_features=n_features)

print(f'CPU VS GPU: Testing with [{n_samples:,.0f}] samples and [{n_features:,.0f}] features !')

# Run UMAP with scikit-learn (CPU)
print('BEGIN [TSNE (sklearn, CPU)] (~30s)...')
start = time.time()
tsne = TSNE(n_components=2)
X_transformed_sklearn = tsne.fit_transform(X)
end = time.time()
print(f"END: TSNE (sklearn, CPU). Execution time: {(end - start):.3f} seconds")

# Run UMAP with cuML (GPU)
print('BEGIN [UMAP (cuML, GPU)] ...')
start = time.time()
umap = UMAP(n_components=2)
X_transformed_cuml = umap.fit_transform(X)
end = time.time()
print(f"END [ UMAP (cuML, GPU)]. Execution time: {(end - start):.3f} seconds")


ImportError: Numba needs NumPy 1.24 or less